# Longest Duplicate Substring

Given a string s, consider all duplicated substrings: (contiguous) substrings of s that occur 2 or more times. The occurrences may overlap.

Return any duplicated substring that has the longest possible length. If s does not have a duplicated substring, the answer is "".

**Example 1:**

Input: s = "banana"
Output: "ana"

**Example 2:**

Input: s = "abcd"
Output: ""

**Constraints:**

- 2 <= s.length <= 3 * 104
- s consists of lowercase English letters.

In [1]:
def build_suffix_array(s):
    """
    Constructs the suffix array for the given string s in O(n log n).
    Using a common 'doubling' method.
    """
    n = len(s)
    # k: current ranking by first 2^k characters
    # suffix array initially sorted by first character
    ranks = [ord(c) for c in s]
    sa = list(range(n))
    k = 0
    while (1 << k) < n:
        # Key for sorting: pair (rank[i], rank[i+2^k]) if exists else -1
        key = lambda x: (ranks[x], ranks[x + (1 << k)] if x + (1 << k) < n else -1)
        sa.sort(key=key)
        
        # temporary array new_ranks to re-rank
        new_ranks = [0]*n
        for i in range(1, n):
            new_ranks[sa[i]] = new_ranks[sa[i-1]]
            if key(sa[i]) != key(sa[i-1]):
                new_ranks[sa[i]] += 1
        ranks = new_ranks
        k += 1
        if ranks[sa[-1]] == n-1:  # all ranks are distinct and sorted
            break
    return sa

def build_lcp(s, sa):
    """
    Construct LCP array using Kasai's algorithm in O(n).
    LCP[i] = length of longest common prefix of sa[i] and sa[i-1].
    """
    n = len(s)
    rank = [0]*n
    for i, pos in enumerate(sa):
        rank[pos] = i
    lcp = [0]*(n)
    h = 0
    for i in range(n):
        if rank[i] > 0:
            j = sa[rank[i]-1]
            while i+h < n and j+h < n and s[i+h] == s[j+h]:
                h += 1
            lcp[rank[i]] = h
            if h > 0:
                h -= 1
    return lcp

def longest_repeated_substring(s):
    if len(s) <= 1:
        return ""
    sa = build_suffix_array(s)
    lcp = build_lcp(s, sa)
    max_lcp = max(lcp)
    if max_lcp == 0:
        return ""
    idx = lcp.index(max_lcp)
    start = sa[idx]
    return s[start:start+max_lcp]

# Example Usage
print(longest_repeated_substring("banana"))  # Expected "ana"
print(longest_repeated_substring("abcd"))    # Expected ""

ana

